In [1]:
import pandas as pd

In [59]:
df_cartola = pd.read_csv('input\\Cartola valorizada.csv')
df_traductor_bodega_sigfe = pd.read_excel('input\\asociacion_bodega_sigfe.xlsx', header = 3)

df_traductor_sigfe_sigcom_cristian = pd.read_excel('input\\Categorías de insumos.xlsx')


df_traductor_bodega_sigfe = df_traductor_bodega_sigfe.rename(columns = {'Nombre Items': \
                                                                        'Item SIGFE'})
TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM = df_traductor_sigfe_sigcom_cristian.iloc[66:83, [0, 1]]
TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM = TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM.rename(columns = 
                                                                {'Destino INT': 'Item SIGFE',
                                                                'Unnamed: 1': 'Item SIGCOM'})

TRADUCTOR_DESTINO_INT_CC_SIGCOM = df_traductor_sigfe_sigcom_cristian.iloc[0:63, [0, 1]]\
                                         .rename(columns = {'Unnamed: 1': 'CC SIGCOM',
                                                            'Destino INT': 'Destino'})

In [60]:
unidas = pd.merge(df_cartola, df_traductor_bodega_sigfe, how = 'left', left_on = 'Codigo Articulo',
                                                                        right_on = 'Código')

unidas_concepto_presup = pd.merge(unidas, TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM, how = 'left',
                                                                            on = 'Item SIGFE')

unidas_destino = pd.merge(unidas_concepto_presup, TRADUCTOR_DESTINO_INT_CC_SIGCOM, how = 'left',
                                                                                   on = 'Destino')

In [83]:
columnas_utiles = ['Codigo Articulo', 'Nombre', 'Movimiento', 'Destino', 'Motivo', 'Familia', 
                   'Item SIGFE', 'Item SIGCOM', 'CC SIGCOM']

In [84]:
df_final = unidas_destino.copy()
df_final = df_final[columnas_utiles]
df_final = df_final.query('Movimiento == "Salida"')

mask_farmacia = ~(df_final['Destino'].str.contains('FARMACIA')) | \
                 (df_final['Destino'].str.contains('SECRE. FARMACIA'))

df_final = df_final[mask_farmacia]
df_final = df_final.query('`Item SIGFE` != "Farmacia"')

In [87]:
motivos_a_filtrar = ['Merma', 'Préstamo', 'Devolución al Proveedor']
df_final = df_final[~df_final['Motivo'].isin(motivos_a_filtrar)]

In [90]:
df_final.Motivo.value_counts()

Despacho                            2591
Despacho Manual - Programado         322
Despacho Manual - Pedido Extra       137
Ajuste                               103
Despacho Manual - Regularización      64
Despacho Manual - Fondo Fijo           9
Otros                                  1
Despacho Manual - Incidencia           1
Name: Motivo, dtype: int64